# AWS_reviews_tfidf_vectorizer_sentiment_analysis

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

# Load and Process Data

In [ ]:
import pandas as pd  # Import the pandas library for data manipulation and analysis

chunk_size = 10000  # Define the size of each chunk to be read from the CSV file

# Read the CSV file in chunks of 'chunk_size' rows at a time
chunks = pd.read_csv("/content/aws_review_sofware_dataset.csv", sep=',', chunksize=chunk_size)

# Retrieve the first chunk from the iterable 'chunks'
df = next(chunks)

# Print the column names of the first chunk
print(df.columns)


Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [ ]:
df.columns  # Display the column names of the DataFrame 'df'

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

# # Data Preprocessing

In [ ]:
# Add a new column 'words' to the DataFrame 'df' and initialize it with the value 'default value'
df["words"] = "default value"

# Add a new column 'sentences' to the DataFrame 'df' and initialize it with the value 'default value'
df["sentences"] = "default value"

# Iterate over each row index in the DataFrame 'df'
for i in range(df.shape[0]):
    # Update the 'words' column of the current row to be an empty list
    df.at[i, "words"] = list("")
    # Update the 'sentences' column of the current row to be an empty list
    df.at[i, "sentences"] = list("")



# Import the function

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Loop through the first 499 rows of the DataFrame 'df'
for i in range(499):
    # Tokenize the text in the 'reviewText' column for the current row into sentences
    l1 = sent_tokenize(df.loc[i, "reviewText"])
    # Assign the list of sentences to the 'sentences' column for the current row
    df.at[i, "sentences"] = l1


In [ ]:
!pip install pywsd

In [ ]:
# Importing the Natural Language Toolkit (nltk) package
import nltk
nltk.download('averaged_perceptron_tagger')  # Downloading the 'averaged_perceptron_tagger' model from nltk's data repository

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# Importing the Natural Language Toolkit (nltk) package
import nltk

# Downloading the 'wordnet' lexical database from nltk
# WordNet is a large lexical database of English, which groups words into sets of synonyms (synsets) and provides definitions and relationships between these words (e.g., antonyms, hyponyms)
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from pywsd.utils import lemmatize_sentence  # Importing the lemmatize_sentence function from pywsd (Python Word Sense Disambiguation)

Warming up PyWSD (takes ~10 secs)... took 10.167060375213623 secs.


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Lemmatization

In [ ]:
# Iterating through each row in the DataFrame
for k in range(df.shape[0]):
    # Initialize the 'words' column as an empty list for the current row
    df.at[k, "words"] = list("")

    # Looping through each sentence in the 'sentences' column for the current row
    for j in range(len(df.loc[k, "sentences"])):
        # Lemmatize the sentence and extend the 'words' list by adding the lemmatized words
        # The lemmatize_sentence function returns a list of words, so we use .extend() to add them to the existing list
        df.at[k, "words"].extend(lemmatize_sentence(df.loc[k, "sentences"][j]))


In [ ]:
# Assigning a default value of "default" to a new column 'words_sentences' in the DataFrame
df["words_sentences"] = "default"


In [ ]:
import functools

# Iterate through the DataFrame
for k in range(df.shape[0]):
    words = df.loc[k, "words"]
    # Check if words is empty and handle accordingly
    if words:
        # Join the words into a single string
        df.loc[k, "words_sentences"] = functools.reduce(lambda a, b: str(a) + " " + str(b), words)
    else:
        # If the list is empty, set the column to an empty string or some default value
        df.loc[k, "words_sentences"] = ""


# TFIDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  # Importing the TfidfVectorizer from the sklearn.feature_extraction.text module

In [ ]:
# Creating an instance of TfidfVectorizer with stop words removed
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# Fitting the vectorizer on the 'words_sentences' column in the DataFrame and transforming it into a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

In [ ]:
# Creating a DataFrame from the dense matrix for better readability and manipulation
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
# Extract the 'verified' column from the DataFrame and assign it to df_y
df_y = df["verified"]


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:

# Create an instance of LabelEncoder
le=LabelEncoder()

In [ ]:
df_y_1=pd.DataFrame(df_y)
# Convert the Series df_y to a DataFrame

In [ ]:
df_y_enc=df_y_1.apply(le.fit_transform)
# Apply LabelEncoder to each column of df_y_1

In [ ]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Initialize Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500, random_state=42)

# Fit the Random Forest model
rf.fit(dense, df_y_enc)

# Accuracy
accuracy_rf = rf.score(dense, df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 64.15%


# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# Import the Multinomial Naive Bayes classifier from Scikit-Learn.

nb = MultinomialNB()
# Initialize an instance of the Multinomial Naive Bayes classifier.

nb.fit(dense, df_y_enc)
# Train the Naive Bayes model using the feature matrix 'dense' and the target labels 'df_y_enc'.

# Compute accuracy
accuracy_nb = nb.score(dense, df_y_enc)
# Calculate the accuracy of the trained model on the training data.

print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")
# Print the accuracy as a percentage, formatted to two decimal places.



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes Accuracy: 63.64%


# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Import the Gradient Boosting Classifier from Scikit-Learn's ensemble module.


In [ ]:
GBC = GradientBoostingClassifier(n_estimators=1000)
# Initialize the Gradient Boosting Classifier with 1000 decision trees (estimators).


In [ ]:
gb_c = GBC.fit(dense, df_y_enc)
# Train the Gradient Boosting Classifier using the feature matrix 'dense' and the target labels 'df_y_enc'.

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Calculate the accuracy of the trained Gradient Boosting Classifier on the training data.
gbc_score=GBC.score(dense, df_y_enc)
print(f"gbc_score: {gbc_score* 100:.2f}%")

gbc_score: 64.15%
